In [1]:
import numpy as np
import pandas as pd
import sklearn
import xgboost as xgb
import matplotlib.pyplot  as plt
import seaborn as sns
import pickle
from datetime import date
from xgboost.sklearn import XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler,LabelEncoder
from sklearn.model_selection import GridSearchCV

In [2]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 500000)
pd.set_option('display.max_rows', 500000)
pd.set_option('display.max_columns', 500000)
pd.set_option('display.width', 900000)

In [3]:
now_year = date.today().strftime('%Y')
now_year

'2019'

In [4]:
df_raw = pd.read_csv('train.csv')
df_raw.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
df_test_raw = pd.read_csv('test.csv')

In [6]:
df_raw.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [ ]:
df_raw.dtypes

In [ ]:
df_raw['Alley']

In [7]:
df_raw['Alley']=df_raw['Alley'].fillna(value='No_Alley')
df_raw['BsmtQual'] = df_raw['BsmtQual'].fillna(value = 'No_Bsmt')
df_raw['BsmtCond'] = df_raw['BsmtCond'].fillna(value = 'No_Bsmt')
df_raw['BsmtExposure'] = df_raw['BsmtExposure'].fillna(value = 'No_Bsmt')
df_raw['BsmtFinType1'] = df_raw['BsmtFinType1'].fillna(value = 'No_Bsmt')
df_raw['BsmtFinType2'] = df_raw['BsmtFinType2'].fillna(value = 'No_Bsmt')
df_raw['FireplaceQu'] = df_raw['FireplaceQu'].fillna(value = 'No_Frplc')
df_raw['GarageType'] = df_raw['GarageType'].fillna(value = 'No_Grge')
df_raw['GarageFinish'] = df_raw['GarageFinish'].fillna(value = 'No_Grge')
df_raw['GarageQual'] = df_raw['GarageQual'].fillna(value = 'No_Grge')
df_raw['GarageCond'] = df_raw['GarageCond'].fillna(value = 'No_Grge')
df_raw['PoolQC'] = df_raw['PoolQC'].fillna(value = 'No_Pool')
df_raw['Fence'] = df_raw['Fence'].fillna(value = 'No_Fnce')
df_raw['MiscFeature'] = df_raw['MiscFeature'].fillna(value = 'None')
df_raw['YearBuilt_noofyrs'] = (pd.Timestamp.now().year - df_raw['YearBuilt'])
df_raw['YearRemodAdd_noofyrs'] = (pd.Timestamp.now().year - df_raw['YearRemodAdd'])
df_raw['GarageYrBlt_noofyrs'] = (pd.Timestamp.now().year - df_raw['GarageYrBlt'])
df_raw['YrSold_noofyrs'] = (pd.Timestamp.now().year - df_raw['YrSold'])
df_raw.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,YearBuilt_noofyrs,YearRemodAdd_noofyrs,GarageYrBlt_noofyrs,YrSold_noofyrs
0,1,60,RL,65.0,8450,Pave,No_Alley,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,No_Frplc,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,No_Pool,No_Fnce,None,0,2,2008,WD,Normal,208500,16,16,16.0,11
1,2,20,RL,80.0,9600,Pave,No_Alley,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,No_Pool,No_Fnce,None,0,5,2007,WD,Normal,181500,43,43,43.0,12
2,3,60,RL,68.0,11250,Pave,No_Alley,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,No_Pool,No_Fnce,None,0,9,2008,WD,Normal,223500,18,17,18.0,11
3,4,70,RL,60.0,9550,Pave,No_Alley,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,No_Pool,No_Fnce,None,0,2,2006,WD,Abnorml,140000,104,49,21.0,13
4,5,60,RL,84.0,14260,Pave,No_Alley,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,No_Pool,No_Fnce,None,0,12,2008,WD,Normal,250000,19,19,19.0,11


In [8]:
df_test_raw['Alley']=df_test_raw['Alley'].fillna(value='No_Alley')
df_test_raw['BsmtQual'] = df_test_raw['BsmtQual'].fillna(value = 'No_Bsmt')
df_test_raw['BsmtCond'] = df_test_raw['BsmtCond'].fillna(value = 'No_Bsmt')
df_test_raw['BsmtExposure'] = df_test_raw['BsmtExposure'].fillna(value = 'No_Bsmt')
df_test_raw['BsmtFinType1'] = df_test_raw['BsmtFinType1'].fillna(value = 'No_Bsmt')
df_test_raw['BsmtFinType2'] = df_test_raw['BsmtFinType2'].fillna(value = 'No_Bsmt')
df_test_raw['FireplaceQu'] = df_test_raw['FireplaceQu'].fillna(value = 'No_Frplc')
df_test_raw['GarageType'] = df_test_raw['GarageType'].fillna(value = 'No_Grge')
df_test_raw['GarageFinish'] = df_test_raw['GarageFinish'].fillna(value = 'No_Grge')
df_test_raw['GarageQual'] = df_test_raw['GarageQual'].fillna(value = 'No_Grge')
df_test_raw['GarageCond'] = df_test_raw['GarageCond'].fillna(value = 'No_Grge')
df_test_raw['PoolQC'] = df_test_raw['PoolQC'].fillna(value = 'No_Pool')
df_test_raw['Fence'] = df_test_raw['Fence'].fillna(value = 'No_Fnce')
df_test_raw['MiscFeature'] = df_test_raw['MiscFeature'].fillna(value = 'None')
df_test_raw['YearBuilt_noofyrs'] = (pd.Timestamp.now().year - df_test_raw['YearBuilt'])
df_test_raw['YearRemodAdd_noofyrs'] = (pd.Timestamp.now().year - df_test_raw['YearRemodAdd'])
df_test_raw['GarageYrBlt_noofyrs'] = (pd.Timestamp.now().year - df_test_raw['GarageYrBlt'])
df_test_raw['YrSold_noofyrs'] = (pd.Timestamp.now().year - df_test_raw['YrSold'])
df_test_raw.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,YearBuilt_noofyrs,YearRemodAdd_noofyrs,GarageYrBlt_noofyrs,YrSold_noofyrs
0,1461,20,RH,80.0,11622,Pave,No_Alley,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,No_Frplc,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,No_Pool,MnPrv,None,0,6,2010,WD,Normal,58,58,58.0,9
1,1462,20,RL,81.0,14267,Pave,No_Alley,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,No_Frplc,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,No_Pool,No_Fnce,Gar2,12500,6,2010,WD,Normal,61,61,61.0,9
2,1463,60,RL,74.0,13830,Pave,No_Alley,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,No_Pool,MnPrv,None,0,3,2010,WD,Normal,22,21,22.0,9
3,1464,60,RL,78.0,9978,Pave,No_Alley,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,No_Pool,No_Fnce,None,0,6,2010,WD,Normal,21,21,21.0,9
4,1465,120,RL,43.0,5005,Pave,No_Alley,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,No_Frplc,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,No_Pool,No_Fnce,None,0,1,2010,WD,Normal,27,27,27.0,9


In [9]:
df_raw.drop(['YearBuilt','YearRemodAdd','GarageYrBlt','YrSold'],axis=1,inplace=True)
df_test_raw.drop(['YearBuilt','YearRemodAdd','GarageYrBlt','YrSold'],axis=1,inplace=True)

In [11]:
numericalData_train=df_raw.select_dtypes(include=np.number)
categoricalData_train=df_raw.select_dtypes(exclude=np.number)

In [12]:
numericalData_test=df_test_raw.select_dtypes(include=np.number)
categoricalData_test=df_test_raw.select_dtypes(exclude=np.number)

In [13]:
fill_NaN = SimpleImputer(missing_values=np.nan, strategy='mean')
clf = fill_NaN.fit(numericalData_train)
imputed_numericalData_train = pd.DataFrame(clf.transform(pd.DataFrame(numericalData_train)))

In [14]:
fill_NaN = SimpleImputer(missing_values=np.nan, strategy='mean')
clf = fill_NaN.fit(numericalData_test)
imputed_numericalData_test = pd.DataFrame(clf.transform(pd.DataFrame(numericalData_test)))

In [15]:
fill_NaN = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='Unknown')
clf = fill_NaN.fit(categoricalData_train)
imputed_categoricalData_train = pd.DataFrame(clf.transform(pd.DataFrame(categoricalData_train)))

In [16]:
fill_NaN = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='Unknown')
clf = fill_NaN.fit(categoricalData_test)
imputed_categoricalData_test = pd.DataFrame(clf.transform(pd.DataFrame(categoricalData_test)))

In [22]:
label_encoder = LabelEncoder()
for x in imputed_categoricalData_train.columns:
    label_encoder.fit(imputed_categoricalData_train[x])
    imputed_categoricalData_train[x] = label_encoder.transform(imputed_categoricalData_train[x])
with open('LECatTransformer.pkl', 'wb') as pkl:
    pickle.dump(label_encoder, pkl)    

In [23]:
with open('LECatTransformer.pkl', 'rb') as pkl: 
    le = pickle.load(pkl)
fitted_classes = list(le.classes_)
for x in imputed_categoricalData_test.columns:
    imputed_categoricalData_test[x]=imputed_categoricalData_test[x].map(lambda x: x in fitted_classes )
    imputed_categoricalData_test[x]=le.transform(imputed_categoricalData_test[x])

In [24]:
imputed_numericalData_df_train = pd.DataFrame(imputed_numericalData_train.values, columns=numericalData_train.columns)
imputed_categoricalData_df_train = pd.DataFrame(imputed_categoricalData_train.values, columns=categoricalData_train.columns)
imputed_df_train = pd.concat([imputed_numericalData_df_train,imputed_categoricalData_df_train], axis=1, sort=False)

In [25]:
imputed_numericalData_df_test = pd.DataFrame(imputed_numericalData_test.values, columns=numericalData_test.columns)
imputed_categoricalData_df_test = pd.DataFrame(imputed_categoricalData_test.values, columns=categoricalData_test.columns)
imputed_df_test = pd.concat([imputed_numericalData_df_test,imputed_categoricalData_df_test], axis=1, sort=False)

In [ ]:
imputed_df_test.head()

In [26]:
plt.scatter(imputed_df.FullBath, imputed_df.SalePrice, marker = "s")
plt.title("FullBath vs SalePrice")
plt.xlabel("FullBath").set_color("white")
plt.ylabel("SalePrice").set_color("white")
plt.show()

NameError: name 'imputed_df' is not defined

In [27]:
X_train = imputed_df_train.drop(['SalePrice','Id'],1)
y_train = imputed_df_train['SalePrice']

In [31]:
Test_ids = imputed_df_test['Id']
imputed_df_test = imputed_df_test.drop(['Id'],1)

In [32]:
X_corr=X_train.corr().abs()
upper = X_corr.where(np.triu(np.ones(X_corr.shape), k=1).astype(np.bool))
drop_cols = [column for column in upper.columns if any(upper[column] > 0.70)]
drop_cols

[]

In [33]:
X_train = X_train[X_train.columns[~X_train.columns.isin(drop_cols)]]
imputed_df_test = imputed_df_test[imputed_df_test.columns[~imputed_df_test.columns.isin(drop_cols)]]

In [34]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YearBuilt_noofyrs,YearRemodAdd_noofyrs,YrSold_noofyrs,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,HouseStyle,RoofStyle,RoofMatl,Exterior1st,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,Fence,MiscFeature,SaleType,SaleCondition
0,60.0,65.0,8450.0,7.0,5.0,196.0,706.0,0.0,150.0,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,0.0,2.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,2.0,16.0,16.0,11.0,3,1,1,3,3,0,4,0,5,2,2,5,1,1,12,1,2,4,2,2,4,3,2,1,0,1,4,2,6,3,1,2,5,5,2,4,1,8,4
1,20.0,80.0,9600.0,6.0,8.0,0.0,978.0,0.0,284.0,1262.0,0.0,0.0,1262.0,0.0,1.0,2.0,0.0,3.0,1.0,1.0,2.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,43.0,43.0,12.0,3,1,1,3,3,0,2,0,24,1,2,2,1,1,8,2,3,4,1,2,4,1,0,1,0,1,4,3,6,5,1,2,5,5,2,4,1,8,4
2,60.0,68.0,11250.0,7.0,5.0,162.0,486.0,0.0,434.0,920.0,866.0,0.0,1786.0,1.0,0.0,2.0,1.0,3.0,1.0,1.0,2.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,9.0,18.0,17.0,11.0,3,1,1,0,3,0,4,0,5,2,2,5,1,1,12,1,2,4,2,2,4,2,2,1,0,1,4,2,6,5,1,2,5,5,2,4,1,8,4
3,70.0,60.0,9550.0,7.0,5.0,0.0,216.0,0.0,540.0,756.0,756.0,0.0,1717.0,1.0,0.0,1.0,0.0,3.0,1.0,1.0,3.0,0.0,35.0,272.0,0.0,0.0,0.0,0.0,2.0,104.0,49.0,13.0,3,1,1,0,3,0,0,0,6,2,2,5,1,1,13,2,3,4,0,4,1,3,0,1,2,1,4,2,6,2,5,3,5,5,2,4,1,8,0
4,60.0,84.0,14260.0,8.0,5.0,350.0,655.0,0.0,490.0,1145.0,1053.0,0.0,2198.0,1.0,0.0,2.0,1.0,4.0,1.0,1.0,3.0,192.0,84.0,0.0,0.0,0.0,0.0,0.0,12.0,19.0,19.0,11.0,3,1,1,0,3,0,2,0,15,2,2,5,1,1,12,1,2,4,2,2,4,0,2,1,0,1,4,2,6,5,1,2,5,5,2,4,1,8,4


In [35]:
imputed_df_test.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YearBuilt_noofyrs,YearRemodAdd_noofyrs,YrSold_noofyrs,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,HouseStyle,RoofStyle,RoofMatl,Exterior1st,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,Fence,MiscFeature,SaleType,SaleCondition
0,20.0,80.0,11622.0,5.0,6.0,0.0,468.0,144.0,270.0,882.0,0.0,0.0,896.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0,1.0,140.0,0.0,0.0,0.0,120.0,0.0,0.0,6.0,58.0,58.0,9.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,20.0,81.0,14267.0,6.0,6.0,108.0,923.0,0.0,406.0,1329.0,0.0,0.0,1329.0,0.0,0.0,1.0,1.0,3.0,1.0,0.0,1.0,393.0,36.0,0.0,0.0,0.0,0.0,12500.0,6.0,61.0,61.0,9.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,60.0,74.0,13830.0,5.0,5.0,0.0,791.0,0.0,137.0,928.0,701.0,0.0,1629.0,0.0,0.0,2.0,1.0,3.0,1.0,1.0,2.0,212.0,34.0,0.0,0.0,0.0,0.0,0.0,3.0,22.0,21.0,9.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,60.0,78.0,9978.0,6.0,6.0,20.0,602.0,0.0,324.0,926.0,678.0,0.0,1604.0,0.0,0.0,2.0,1.0,3.0,1.0,1.0,2.0,360.0,36.0,0.0,0.0,0.0,0.0,0.0,6.0,21.0,21.0,9.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,120.0,43.0,5005.0,8.0,5.0,0.0,263.0,0.0,1017.0,1280.0,0.0,0.0,1280.0,0.0,0.0,2.0,0.0,2.0,1.0,0.0,2.0,0.0,82.0,0.0,0.0,144.0,0.0,0.0,1.0,27.0,27.0,9.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [36]:
xgb1 = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train,
         y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  18 out of  18 | elapsed:   14.4s finished
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


0.8758580116693592
{'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 5, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}


In [37]:
parameters = {'nthread':4, #when use hyperthread, xgboost may become slower
              'objective':'reg:linear',
              'learning_rate':  0.05, #so called `eta` value
              'max_depth': 5,
              'min_child_weight': 4,
              'silent': 1,
              'subsample': 0.7,
              'colsample_bytree': 0.7,
              'n_estimators': 500}
xgb1 = XGBRegressor(max_depth=5,learning_rate=0.05,n_estimators=500,min_child_weight=4,silent=1,subsample=0.7,colsample_bytree=0.7,objective='reg:linear')
xgb1.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.7, gamma=0,
       importance_type='gain', learning_rate=0.05, max_delta_step=0,
       max_depth=5, min_child_weight=4, missing=None, n_estimators=500,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=1,
       subsample=0.7, verbosity=1)

In [38]:
feature_importances = pd.DataFrame(xgb1.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)

In [39]:
newfeatures=feature_importances[feature_importances.importance<=0]
newfeatures=newfeatures.rename(columns={'index':'columnname'})
newfeatures.reset_index(inplace=True)
newfeatures_cols = list(newfeatures['index'])

In [40]:
X_train=X_train[X_train.columns[~X_train.columns.isin(newfeatures_cols)]]
imputed_df_test = imputed_df_test[imputed_df_test.columns[~imputed_df_test.columns.isin(newfeatures_cols)]]

In [41]:
xgb1.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.7, gamma=0,
       importance_type='gain', learning_rate=0.05, max_delta_step=0,
       max_depth=5, min_child_weight=4, missing=None, n_estimators=500,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=1,
       subsample=0.7, verbosity=1)

In [42]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YearBuilt_noofyrs,YearRemodAdd_noofyrs,YrSold_noofyrs,MSZoning,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,HouseStyle,RoofStyle,RoofMatl,Exterior1st,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,Fence,MiscFeature,SaleType,SaleCondition
0,60.0,65.0,8450.0,7.0,5.0,196.0,706.0,0.0,150.0,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,0.0,2.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,2.0,16.0,16.0,11.0,3,1,3,3,4,0,5,2,2,5,1,1,12,1,2,4,2,2,4,3,2,1,0,1,4,2,6,3,1,2,5,5,2,4,1,8,4
1,20.0,80.0,9600.0,6.0,8.0,0.0,978.0,0.0,284.0,1262.0,0.0,0.0,1262.0,0.0,1.0,2.0,0.0,3.0,1.0,1.0,2.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,43.0,43.0,12.0,3,1,3,3,2,0,24,1,2,2,1,1,8,2,3,4,1,2,4,1,0,1,0,1,4,3,6,5,1,2,5,5,2,4,1,8,4
2,60.0,68.0,11250.0,7.0,5.0,162.0,486.0,0.0,434.0,920.0,866.0,0.0,1786.0,1.0,0.0,2.0,1.0,3.0,1.0,1.0,2.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,9.0,18.0,17.0,11.0,3,1,0,3,4,0,5,2,2,5,1,1,12,1,2,4,2,2,4,2,2,1,0,1,4,2,6,5,1,2,5,5,2,4,1,8,4
3,70.0,60.0,9550.0,7.0,5.0,0.0,216.0,0.0,540.0,756.0,756.0,0.0,1717.0,1.0,0.0,1.0,0.0,3.0,1.0,1.0,3.0,0.0,35.0,272.0,0.0,0.0,0.0,0.0,2.0,104.0,49.0,13.0,3,1,0,3,0,0,6,2,2,5,1,1,13,2,3,4,0,4,1,3,0,1,2,1,4,2,6,2,5,3,5,5,2,4,1,8,0
4,60.0,84.0,14260.0,8.0,5.0,350.0,655.0,0.0,490.0,1145.0,1053.0,0.0,2198.0,1.0,0.0,2.0,1.0,4.0,1.0,1.0,3.0,192.0,84.0,0.0,0.0,0.0,0.0,0.0,12.0,19.0,19.0,11.0,3,1,0,3,2,0,15,2,2,5,1,1,12,1,2,4,2,2,4,0,2,1,0,1,4,2,6,5,1,2,5,5,2,4,1,8,4


In [43]:
print(list(X_train.columns))

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'Fireplaces', 'GarageCars', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YearBuilt_noofyrs', 'YearRemodAdd_noofyrs', 'YrSold_noofyrs', 'MSZoning', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [44]:
print(list(imputed_df_test.columns))

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'Fireplaces', 'GarageCars', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YearBuilt_noofyrs', 'YearRemodAdd_noofyrs', 'YrSold_noofyrs', 'MSZoning', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [45]:
imputed_df_test.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YearBuilt_noofyrs,YearRemodAdd_noofyrs,YrSold_noofyrs,MSZoning,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,HouseStyle,RoofStyle,RoofMatl,Exterior1st,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,Fence,MiscFeature,SaleType,SaleCondition
0,20.0,80.0,11622.0,5.0,6.0,0.0,468.0,144.0,270.0,882.0,0.0,0.0,896.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0,1.0,140.0,0.0,0.0,0.0,120.0,0.0,0.0,6.0,58.0,58.0,9.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,20.0,81.0,14267.0,6.0,6.0,108.0,923.0,0.0,406.0,1329.0,0.0,0.0,1329.0,0.0,0.0,1.0,1.0,3.0,1.0,0.0,1.0,393.0,36.0,0.0,0.0,0.0,0.0,12500.0,6.0,61.0,61.0,9.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,60.0,74.0,13830.0,5.0,5.0,0.0,791.0,0.0,137.0,928.0,701.0,0.0,1629.0,0.0,0.0,2.0,1.0,3.0,1.0,1.0,2.0,212.0,34.0,0.0,0.0,0.0,0.0,0.0,3.0,22.0,21.0,9.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,60.0,78.0,9978.0,6.0,6.0,20.0,602.0,0.0,324.0,926.0,678.0,0.0,1604.0,0.0,0.0,2.0,1.0,3.0,1.0,1.0,2.0,360.0,36.0,0.0,0.0,0.0,0.0,0.0,6.0,21.0,21.0,9.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,120.0,43.0,5005.0,8.0,5.0,0.0,263.0,0.0,1017.0,1280.0,0.0,0.0,1280.0,0.0,0.0,2.0,0.0,2.0,1.0,0.0,2.0,0.0,82.0,0.0,0.0,144.0,0.0,0.0,1.0,27.0,27.0,9.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [46]:
y_pred = xgb1.predict(imputed_df_test)

In [47]:
y_pred

array([116422.18 , 169005.9  , 190033.5  , ..., 195247.69 , 124131.984,
       230454.23 ], dtype=float32)

In [102]:
Y_pred_df=pd.DataFrame(y_pred)

In [103]:
Y_pred_df.head()
Y_pred_df=Y_pred_df.rename(columns={0:'SalePrice'})

In [104]:
Y_pred_df = Y_pred_df.merge(Test_ids,how='left',left_index=True, right_index=True)

In [108]:
Y_pred_df.to_csv('Kaggle_AdvancedRegression_Submission.csv',index=False)

In [92]:
Y_pred_df['Id'].astype('int32').dtypes

dtype('int32')

In [106]:
Y_pred_df.Id = Y_pred_df.Id.astype(int)

In [107]:
Y_pred_df.head()

,SalePrice,Id
0,116422.179688,1461
1,169005.906250,1462
2,190033.500000,1463
3,196791.718750,1464
4,184431.671875,1465
